In [7]:
import os
import glob
import json
import numpy as np
import pandas as pd

from brave import brave
from convert_srl_to_brat import SrlConverter
from brave import brave, start_notebook_mode, brave_compare

from isanlp_srl_framebank.pipeline_default import PipelineDefault

In [8]:
start_notebook_mode()

In [9]:
ppl = PipelineDefault(address_morph=('localhost', 3333),
                      address_syntax=('localhost', 3334),
                      address_srl=('localhost', 3335))

converter = SrlConverter()

In [10]:
sent = "На этих вечеринках хозяйка и гости не играли в карты и не танцевали а развлекали себя разными художествами."
sent = "В тихую июльскую ночь Ольга Ивановна стояла на палубе волжского парохода и смотрела то на воду то на красивые берега."
sent = "В старших классах на дополнительных уроках мы читали вслух Гоголя или Пушкина или писали сочинения."
sent = "С самого утра в тот памятный день ярко и жизнерадостно светило солнце и воробьи весело чирикали о чём-то друг с другом."
sent = "Я не умею ни красиво говорить ни спорить ни поддерживать разговора."

sent = "Дул то с одной то с другой стороны ветерок."
sent = "По вечерам я стал читать и оценил вполне лампу над столом и седые угольки на подносе самовара и стынущий чай."
sent = "На широком лугу пасутся коровы да лошади или козы с овцами."
sent = "У Пахомки ни детей ни воспитанников никогда не было."

sent = "В воздухе жужжали мухи и наперегонки носились разноцветные бабочки."
sent = "Потом родители умерли и Робин остался один."
sent = "Вечером стало смеркаться и мы заметили молодого оленя."

sent = "И санки и игрушки ей дарили и она уже было поверила в то, что её любят."
sent = "У нашего Робина будет и прекрасная невеста и половина царства."
sent = "Приходили то родственники то друзья то совершенно посторонние люди."
sent = "Или вот сидишь за столом что-нибудь выпил или съел и в животе как заурчит и кто-нибудь на тебя так посмотрит дескать ну как так можно…"
sent = "Солнце то выглядывало то скрывалось за пушистыми облаками."
sent = "От восторга тот вроде бы запьянел на миг и рогатая голова пошла кругом."
sent = "Но Ёжик открыл люк спустился в погреб достал и грибков и брусники и орехов - всего что было у них в доме."
sent = "В шкафу и на столе оказались фрукты и разнообразные сладости."
sent = "Такой сотрудник будет не только ежемесячно получать оклад и премии но и подниматься по карьерной лестнице."
sent = "После каждой поездки в горы у меня появлялись новые акварельные этюды или множество карандашных набросков или же подробные записи о пережитых впечатлениях."

In [11]:
%%time

annotations = ppl(sent)

CPU times: user 7.34 ms, sys: 7.48 ms, total: 14.8 ms
Wall time: 1.1 s


In [12]:
brat_json = converter(annotations)

coll_data = brat_json['coll_data']
doc_data = brat_json['doc_data']

In [13]:
brave(doc_data, coll_data)

# Annotations:

In [14]:
links = []

for i_, (id_, postag_, ws_) in enumerate(zip(sent.split(), annotations['postag'][0], annotations['syntax_dep_tree'][0])):
    print(i_, id_, ws_.link_name, postag_, ws_.parent)
    links.append(ws_.link_name)

0 После case ADP 2
1 каждой det PRON 2
2 поездки nmod NOUN 7
3 в case ADP 4
4 горы nmod NOUN 2
5 у case ADP 6
6 меня nmod PRON 7
7 появлялись ROOT VERB -1
8 новые amod ADJ 10
9 акварельные amod ADJ 10
10 этюды nsubj NOUN 7
11 или cc CONJ 10
12 множество conj NOUN 10
13 карандашных amod ADJ 14
14 набросков nmod NOUN 12
15 или cc CONJ 10
16 же advmod PART 15
17 подробные amod ADJ 18
18 записи conj NOUN 10
19 о case ADP 21
20 пережитых amod VERB 21
21 впечатлениях. nmod NOUN 18


In [15]:
from collections import Counter

In [16]:
Counter(links)

Counter({'case': 4,
         'det': 1,
         'nmod': 5,
         'ROOT': 1,
         'amod': 5,
         'nsubj': 1,
         'cc': 2,
         'conj': 2,
         'advmod': 1})

In [17]:
# 1 стало ROOT -1
# 2 смеркаться nsubj 1
# 3 и cc 1
# 4 мы nsubj 5
# 5 заметили conj 1

In [18]:
# 1 родители nsubj 2
# 2 умерли ROOT -1
# 3 и advmod 4
# 4 Робин dobj 2
# 5 остался conj 2

In [19]:
# 2 жужжали ROOT -1
# 3 мухи nsubj 2
# 4 и cc 2
# 6 носились conj 2
# 8 бабочки. nsubj 6

# Classifier:

In [20]:
# !pip install -U scikit-learn

In [58]:
import re
import os
import glob
import json
import numpy as np
import pandas as pd

from lightgbm import LGBMClassifier

In [59]:
TEST_FOLDER = "/root/shared/ai-journey-2019/data/reshu_ege/task_16/"

In [60]:
tests_list = glob.glob(os.path.join(TEST_FOLDER, "*.json"))

In [61]:
tests = []
tests_dfs = []

for test_path in tests_list:
    test_ = json.load(open(test_path))
    tests.append(test_)
    tests_dfs.append(pd.DataFrame.from_dict(test_['tasks']))

In [62]:
df_tests = pd.concat(tests_dfs)

In [63]:
# 

In [80]:
# all_sentences = df_tests.question.astype(str).apply(eval).apply(lambda x: x.get("choices")).values
# all_correct_answers = df_tests.solution.astype(str).apply(eval).apply(lambda x: x[list(x.keys())[0]]).values

In [67]:
# list(map(list, all_sentences))

In [112]:
sentences_pos = []
sentences_neg = []

for test_ in tests:
    sentences_ = list(map(lambda x: x['text'], test_['tasks'][0]['question']['choices']))

    if 'correct_variants' in test_['tasks'][0]['solution']:
        answers_ = set(np.array(test_['tasks'][0]['solution']['correct_variants'][0]).astype(int))
    else:
        answers_ = set(np.array(test_['tasks'][0]['solution']['correct']).astype(int))
    
    answers_ = set(np.array(list(answers_)) - 1)
    
    sentences_pos.append(list(np.array(sentences_)[list(answers_)]))
    sentences_neg.append(list(np.array(sentences_)[list(set(list(range(len(sentences_)))).difference(answers_))]))

In [270]:
all_pos_sentences = list(np.concatenate(sentences_pos))
all_neg_sentences = list(np.concatenate(sentences_neg))

In [214]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [144]:
corpus = all_pos_sentences + all_neg_sentences

In [258]:
vectorizer = TfidfVectorizer(max_features=1000, analyzer='char', ngram_range=(1, 10))
# vectorizer = CountVectorizer(max_features=1000, analyzer='char', ngram_range=(1, 5))

In [259]:
X = vectorizer.fit_transform(corpus)
y = np.concatenate([np.ones(len(all_pos_sentences)), 
                    np.zeros(len(all_neg_sentences))])

In [278]:
clf = LGBMClassifier()

In [279]:
from sklearn.model_selection import cross_val_predict

In [280]:
y_pred = cross_val_predict(clf, X.astype(float), y, cv=10, method='predict')

In [281]:
from sklearn.metrics import accuracy_score, log_loss

In [284]:
accuracy_score(y, y_pred), accuracy_score(y, np.zeros_like(y))

(0.7162011173184357, 0.6)

## PoS tags:

In [1743]:
import pymorphy2

from tqdm import tqdm_notebook as tqdm

In [765]:
corpus = all_pos_sentences + all_neg_sentences

In [766]:
# corpus_annots = [ppl(str(sent_)) for sent_ in tqdm(corpus)]

In [767]:
corpus_ws = [" ".join(list(map(lambda x: x.link_name, annot_['syntax_dep_tree'][0]))) for annot_ in corpus_annots]

corpus_pos = [" ".join(annot_['postag'][0]) for annot_ in corpus_annots]

In [1355]:
morph = pymorphy2.MorphAnalyzer()

In [1388]:
corpus_pos_pymorph = [" ".join(list(filter(lambda x: x,
                                        [morph.parse(word_)[0].tag.POS for word_ in sentence_.strip(".").split()]))) 
                                                  for sentence_ in corpus]

In [1727]:
# vectorizer_ws = TfidfVectorizer(max_features=90, analyzer='word', ngram_range=(1, 15))
vectorizer_pos = CountVectorizer(max_features=30, analyzer='word', ngram_range=(1, 4))
vectorizer_words = CountVectorizer(max_features=90, analyzer='word', ngram_range=(1, 4))

In [1728]:
# X_ws = vectorizer_ws.fit_transform(corpus_ws)
X_pos = vectorizer_pos.fit_transform(corpus_pos_pymorph)
X_words = vectorizer_words.fit_transform(corpus)

y = np.concatenate([np.ones(len(all_pos_sentences)), 
                    np.zeros(len(all_neg_sentences))])

In [1748]:
X_full = np.concatenate([
#     X_ws.astype(float).toarray(),
    X_pos.astype(float).toarray(), 
    X_words.astype(float).toarray()], axis=1)

# X_full = X_pos.astype(float).toarray()

In [1749]:
clf = LGBMClassifier(n_estimators=65)

In [1750]:
y_pred_full = cross_val_predict(clf, X_full, y, cv=10, method='predict')

In [1751]:
clf.fit(X_full, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=65, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [1752]:
accuracy_score(y, y_pred_full), accuracy_score(y, y_pred), accuracy_score(y, np.zeros_like(y))

(0.7452513966480447, 0.7162011173184357, 0.6)

In [1753]:
# y_pred_full

### Save models:

In [1754]:
# Save to file in the directory
pkl_filename = "../models/task_16/task_16_clf.pkl"

with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)
    
pkl_filename = "../models/task_16/task_16_vectorizer_words.pkl"

with open(pkl_filename, 'wb') as file:
    pickle.dump(vectorizer_words, file)
    
pkl_filename = "../models/task_16/task_16_vectorizer_pos.pkl"

with open(pkl_filename, 'wb') as file:
    pickle.dump(vectorizer_pos, file)

# Load full solution:

In [1755]:
import pickle
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [1756]:
pkl_filename = "../models/task_16/task_16_clf.pkl"

with open(pkl_filename, 'rb') as file:
    clf = pickle.load(file)

pkl_filename = "../models/task_16/task_16_vectorizer_words.pkl"

with open(pkl_filename, 'rb') as file:
    vectorizer_words = pickle.load(file)
    
pkl_filename = "../models/task_16/task_16_vectorizer_pos.pkl"

with open(pkl_filename, 'rb') as file:
    vectorizer_pos = pickle.load(file)

In [1757]:
def embedder(sentence):
#     annot_ = ppl(str(sentence))
#     ws_sentence = " ".join(list(map(lambda x: x.link_name, annot_['syntax_dep_tree'][0])))
#     pos_sentence = " ".join(annot_['postag'][0])
    pos_sentence = " ".join(list(filter(lambda x: x,
                                [morph.parse(word_)[0].tag.POS for word_ in sentence.strip(".").split()])))
    
#     x_ws = vectorizer_ws.transform([ws_sentence])
    x_pos = vectorizer_pos.transform([pos_sentence])
    x_word = vectorizer_words.transform([str(sentence)])
    
    x = np.concatenate([x_pos.toarray(), x_word.toarray()], axis=1)
    
    return x

In [1782]:
def predict_sentences_task_16(list_of_sentences):
    
    def _get_2_sentences(probas):
        probas_positions = np.argsort(probas) + 1
        return list(probas_positions[:2])
    
    probas = []

    for sent_ in list_of_sentences:
        x = embedder(sent_)
        proba = clf.predict_proba(x)
        probas.append(proba)
        
    sent_probas = np.concatenate(probas)[:, 0]
    
    return _get_2_sentences(sent_probas)

In [1779]:
def solver_16(task):
    sentences = list(map(lambda x: x['text'], task['question']['choices']))
    
    return np.array(predict_sentences_task_16(sentences)).astype(str).tolist()

In [1784]:
%%time

predict_sentences(sample_sentences)

CPU times: user 388 ms, sys: 3.98 ms, total: 392 ms
Wall time: 39.5 ms


[2, 3]

# Validation:

In [1698]:
TEST_FOLDER = "/root/shared/ai-journey-2019/data/check/"

In [1699]:
tests_list = glob.glob(os.path.join(TEST_FOLDER, "*.json"))

In [1700]:
tests = []
tests_dfs = []

for test_path in tests_list:
    test_ = json.load(open(test_path))
    tests.append(test_)
    tests_dfs.append(pd.DataFrame.from_dict(test_['tasks']))

In [1701]:
answers = []
sentences = []

for test_ in tests:
    sentences_ = list(map(lambda x: x['text'], test_['tasks'][15]['question']['choices']))

    if 'correct_variants' in test_['tasks'][15]['solution']:
        answers_ = set(np.array(test_['tasks'][15]['solution']['correct_variants'][0]).astype(int))
    else:
        answers_ = set(np.array(test_['tasks'][15]['solution']['correct']).astype(int))
    
    answers_ = set(np.array(list(answers_)))
    
    answers.append(list(answers_))
    sentences.append(sentences_)

# check:

In [1702]:
results = []

for sentences_pack in tqdm(sentences, position=0):
    results.append(predict_sentences(sentences_pack))